In [1]:
import numpy as np 
import pandas as pd

from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LogisticRegression
from sklearn import metrics

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/cis-ieee-unb-renting-houce-prices/train.csv
/kaggle/input/cis-ieee-unb-renting-houce-prices/test.csv


In [2]:
#Concatena os datasets. A ideia é fazer o préprocessamento com todos os dados e depois separar.

df_train = pd.read_csv('/kaggle/input/cis-ieee-unb-renting-houce-prices/train.csv')
df_test = pd.read_csv('/kaggle/input/cis-ieee-unb-renting-houce-prices/test.csv')

# Adicionar a coluna 'is_train'
df_train['is_train'] = 1
df_test['is_train'] = 0

# Concatenar os dataframes
df = pd.concat([df_train, df_test], axis=0)
del df_train
del df_test
df.reset_index(drop=True, inplace=True)
df.head()

/tmp/ipykernel_235/1692342216.py:3: DtypeWarning: Columns (0,21,24,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train = pd.read_csv('/kaggle/input/cis-ieee-unb-renting-houce-prices/train.csv')


,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,...,extra_people,minimum_nights,number_of_reviews,instant_bookable,amenities,property_type,room_type,cancellation_policy,price,is_train
0,947924,https://www.airbnb.com/rooms/947924,2.018082e+13,2018-08-16,LARGE LOVELY ROOM GREAT FOR GROUPS,NaN,This large lovely room is perfect for any grou...,This large lovely room is perfect for any grou...,none,NaN,...,$82.00,1,4.0,f,"{Internet,Wifi,Kitchen,""Free parking on premis...",House,Private room,strict_14_with_grace_period,$229.00,1
1,2747040,https://www.airbnb.com/rooms/2747040,2.019072e+13,2019-07-16,Alugo para a Copa do mundo,"Excelente apto, localizado a 500 metros de dis...",NaN,"Excelente apto, localizado a 500 metros de dis...",none,NaN,...,$0.00,10,0.0,f,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",Apartment,Entire home/apt,moderate,"$1,502.00",1
2,13734316,https://www.airbnb.com/rooms/13734316,2.018112e+13,2018-11-15,Apartamento para temporada,Meu espaço é perto de Vila olímpica dos atleta...,NaN,Meu espaço é perto de Vila olímpica dos atleta...,none,NaN,...,$0.00,30,0.0,t,"{TV,""Air conditioning"",Pool,Kitchen,""Free park...",Apartment,Entire home/apt,flexible,$569.00,1
3,30979175,https://www.airbnb.com/rooms/30979175,2.019092e+13,2019-09-24,2-Ensuite flat with stunning sea view!,Enjoy the most beautiful sight of Recreio dos ...,The flat is newly refurbished and has a modern...,Enjoy the most beautiful sight of Recreio dos ...,none,NaN,...,$75.00,2,20.0,f,"{TV,""Cable TV"",Wifi,""Air conditioning"",Pool,Ki...",Apartment,Entire home/apt,moderate,$499.00,1
4,3742926,https://www.airbnb.com/rooms/3742926,2.018121e+13,2018-12-14,Condomino,"Rio-Stadtteil JACAREPAGUA-CURICICA, kein Touri...",NaN,"Rio-Stadtteil JACAREPAGUA-CURICICA, kein Touri...",none,NaN,...,$88.00,7,0.0,f,"{TV,""Cable TV"",Internet,Wifi,""Air conditioning...",House,Private room,flexible,$132.00,1


In [3]:
#Remove as colunas irrelevantes para analise de preço

irrelevant_cols = ['id', 
                   'listing_url',
                   'scrape_id',
                   'last_scraped',
                   'name',
                   'summary',
                   'space',
                   'description',
                   'experiences_offered',
                   'neighborhood_overview',
                   'amenities',
                   'property_type'#Temporariamente
                  ]

df = df.drop(columns=irrelevant_cols)

In [4]:
#Elimina valores que não são numéricos de colunas que deveriam ser numéricas
categorical_cols = ['host_is_superhost',
                    'instant_bookable',
                    'property_type',
                    'room_type',
                    'cancellation_policy']

cols = list(df.columns)
numerical_cols = list(set(cols) - set(categorical_cols))


df['extra_people'] = df['extra_people'].replace('[\$,]', '', regex=True)
df['price'] = df['price'].replace('[\$,]', '', regex=True)

for col in numerical_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
    

df.info()   

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784121 entries, 0 to 784120
Data columns (total 26 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   minimum_minimum_nights                        600998 non-null  float64
 1   maximum_minimum_nights                        600998 non-null  float64
 2   minimum_maximum_nights                        600998 non-null  float64
 3   maximum_maximum_nights                        600998 non-null  float64
 4   minimum_nights_avg_ntm                        600998 non-null  float64
 5   maximum_nights_avg_ntm                        600998 non-null  float64
 6   number_of_reviews_ltm                         600998 non-null  float64
 7   calculated_host_listings_count_entire_homes   600998 non-null  float64
 8   calculated_host_listings_count_private_rooms  600998 non-null  float64
 9   calculated_host_listings_count_shared_rooms   60

In [5]:
#Verifica cada valor único das colunas para ver se tem valores potencialmente "errados"
categorical_cols = ['host_is_superhost',
                    'instant_bookable',
                    #'property_type',
                    'room_type',
                    'cancellation_policy']


for col in categorical_cols:
    print(df[col].value_counts().to_string())
    print()

host_is_superhost
f    697930
t     85805
3         1

instant_bookable
f    480254
t    303866

room_type
Entire home/apt    555541
Private room       208024
Shared room         18219
Hotel room           2336
2019-03-06              1

cancellation_policy
flexible                       335621
strict_14_with_grace_period    306662
moderate                       131093
super_strict_60                  7843
super_strict_30                  2857
strict                             44



In [6]:
#Remove alguns valores que no dataset que aparentam ter sido preenchidos incorretamente
df['host_is_superhost'] = df['host_is_superhost'].replace('3', np.nan)
#df['property_type'] = df['property_type'].replace('1', np.nan)
df['room_type'] = df['room_type'].replace('2019-03-06', np.nan)

In [7]:
#Imputa a moda em colunas categoricas com valores faltantes e média em colunas numéricas
categorical_cols = ['host_is_superhost',
                    'instant_bookable',
                    #'property_type',
                    'room_type',
                    'cancellation_policy']

for col in categorical_cols:
    mode_value = df[col].mode()[0]
    df[col] = df[col].fillna(mode_value)
    
cols = list(df.columns)
numerical_cols = list(set(cols) - set(categorical_cols))

mean_values = df[numerical_cols].mean()
df[numerical_cols] = df[numerical_cols].fillna(mean_values)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784121 entries, 0 to 784120
Data columns (total 26 columns):
 #   Column                                        Non-Null Count   Dtype  
---  ------                                        --------------   -----  
 0   minimum_minimum_nights                        784121 non-null  float64
 1   maximum_minimum_nights                        784121 non-null  float64
 2   minimum_maximum_nights                        784121 non-null  float64
 3   maximum_maximum_nights                        784121 non-null  float64
 4   minimum_nights_avg_ntm                        784121 non-null  float64
 5   maximum_nights_avg_ntm                        784121 non-null  float64
 6   number_of_reviews_ltm                         784121 non-null  float64
 7   calculated_host_listings_count_entire_homes   784121 non-null  float64
 8   calculated_host_listings_count_private_rooms  784121 non-null  float64
 9   calculated_host_listings_count_shared_rooms   78

In [8]:
#Tentativa de reduzir a quantidade de memória utilizada
colunas_float64 = df.select_dtypes(include=['float64']).columns
df[colunas_float64] = df[colunas_float64].astype('float32')

In [9]:
# Importar a biblioteca PCA
from sklearn.decomposition import PCA

# Aplica one-hot-encoding em colunas categóricas
ohe = OneHotEncoder(sparse_output=False, dtype=np.int32)
one_hot_encoded = pd.DataFrame(ohe.fit_transform(df[categorical_cols]), columns=ohe.get_feature_names_out(categorical_cols))
df = pd.concat([df, one_hot_encoded], axis=1)
df = df.drop(columns=categorical_cols)
print(df.info())

# Remover a coluna 'is_train' temporariamente para aplicar o PCA
is_train = df.pop('is_train')

price = df.pop('price')

pca = PCA(n_components=5)  
pca_result = pca.fit_transform(df)

df_pca = pd.DataFrame(pca_result)

df_pca = pd.DataFrame(pca_result)

# Recolocaa as colunas 'is_train' e 'price' no DataFrame resultante
df_pca['is_train'] = is_train
df_pca['price'] = price.values  # Adicionar os valores de volta como uma nova coluna

# Separação do dataset
df_train = df_pca[df_pca['is_train'] == 1].copy()
df_test = df_pca[df_pca['is_train'] == 0].copy()

X_train = df_train.drop(['is_train', 'price'], axis=1)
Y_train = df_train['price']

X_test = df_test.drop(['is_train'], axis=1)
Y_test = df_test['price']

# Remover df intermediário
del df_pca
del df_train
del df_test

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 784121 entries, 0 to 784120
Data columns (total 36 columns):
 #   Column                                           Non-Null Count   Dtype  
---  ------                                           --------------   -----  
 0   minimum_minimum_nights                           784121 non-null  float32
 1   maximum_minimum_nights                           784121 non-null  float32
 2   minimum_maximum_nights                           784121 non-null  float32
 3   maximum_maximum_nights                           784121 non-null  float32
 4   minimum_nights_avg_ntm                           784121 non-null  float32
 5   maximum_nights_avg_ntm                           784121 non-null  float32
 6   number_of_reviews_ltm                            784121 non-null  float32
 7   calculated_host_listings_count_entire_homes      784121 non-null  float32
 8   calculated_host_listings_count_private_rooms     784121 non-null  float32
 9   calculated_host

In [ ]:
#Treina o modelo
model = LogisticRegression()
model.fit(X_train, Y_train)

y_pred = model.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)

print(f'Acurácia: {accuracy:.2f}')

print(metrics.classification_report(y_test, y_pred))